In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import mode


from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical

from matplotlib import ticker
import time

In [2]:
train = pd.read_csv("E:/Fear/Univ/Big_data/Training/python/tabular-playground-series-dec-2021/train.csv")
test = pd.read_csv("E:/Fear/Univ/Big_data/Training/python/tabular-playground-series-dec-2021/test.csv")
submission = pd.read_csv("E:/Fear/Univ/Big_data/Training/python/tabular-playground-series-dec-2021/sample_submission.csv")

In [3]:
train.drop(["Id"] , axis = 1 , inplace = True)
test.drop(["Id"] , axis = 1 , inplace = True)
TARGET = 'Cover_Type'
FEATURES = [col for col in train.columns if col not in ['id', TARGET]]
RANDOM_STATE = 12 

In [4]:
train = train.drop(index = int(np.where(train["Cover_Type"] == 5 )[0]))
train = train.drop(train[train["Cover_Type"] == 4].index[0])

#train = train.reset_index(drop=True)

In [5]:
train = train.drop(labels = ["Soil_Type7" , "Soil_Type15", "Soil_Type8", "Soil_Type25"] ,axis = 1)
FEATURES.remove('Soil_Type7')
FEATURES.remove('Soil_Type8')
FEATURES.remove('Soil_Type15')
FEATURES.remove('Soil_Type25')

In [6]:
scaler = StandardScaler()
for col in FEATURES:
    train[col] = scaler.fit_transform(train[col].to_numpy().reshape(-1,1))
    test[col] = scaler.transform(test[col].to_numpy().reshape(-1,1))
    
X = train[FEATURES].to_numpy().astype(np.float32)
y = train[TARGET].to_numpy().astype(np.float32)
X_test = test[FEATURES].to_numpy().astype(np.float32)

del train, test

In [ ]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 2048
EPOCHS = 100
VALIDATION_RATIO = 0.05

LE = LabelEncoder()
y = to_categorical(LE.fit_transform(y))
X_train , X_valid ,y_train ,y_valid  = train_test_split(X,y , test_size = VALIDATION_RATIO , random_state=RANDOM_STATE)


def load_model(): 
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation = 'swish', input_shape = [X.shape[1]]),
        tf.keras.layers.Dense(1024, activation ='swish'),
        tf.keras.layers.Dense(512, activation ='swish'),
        tf.keras.layers.Dense(6, activation='softmax'),
    ])
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['acc'],
    )
    return model
    
    
early_stopping = callbacks.EarlyStopping(
        patience=10,
        min_delta=0,
        monitor='val_loss',
        restore_best_weights=True,
        verbose=0,
        mode='min', 
        baseline=None,
    )
plateau = callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2, 
            patience=4, 
            verbose=0,
            mode='min')

nn_model = load_model()
history = nn_model.fit(  X_train , y_train,
                validation_data = (X_valid , y_valid),
                batch_size = BATCH_SIZE, 
                epochs = EPOCHS,
                callbacks = [early_stopping , plateau],
              )
nn_preds = nn_model.predict(X_test , batch_size=BATCH_SIZE)

Epoch 1/100
1856/1856 [==============================] - 339s 182ms/step - loss: 0.1899 - acc: 0.9338 - val_loss: 0.1296 - val_acc: 0.9489 - lr: 1.0000e-04
Epoch 2/100
1856/1856 [==============================] - 337s 181ms/step - loss: 0.1194 - acc: 0.9513 - val_loss: 0.1134 - val_acc: 0.9532 - lr: 1.0000e-04
Epoch 3/100
1856/1856 [==============================] - 333s 179ms/step - loss: 0.1061 - acc: 0.9548 - val_loss: 0.1030 - val_acc: 0.9561 - lr: 1.0000e-04
Epoch 4/100
1856/1856 [==============================] - 332s 179ms/step - loss: 0.0992 - acc: 0.9569 - val_loss: 0.0980 - val_acc: 0.9570 - lr: 1.0000e-04
Epoch 5/100
1856/1856 [==============================] - 331s 179ms/step - loss: 0.0948 - acc: 0.9581 - val_loss: 0.0950 - val_acc: 0.9580 - lr: 1.0000e-04
Epoch 6/100
1856/1856 [==============================] - 342s 184ms/step - loss: 0.0917 - acc: 0.9590 - val_loss: 0.0918 - val_acc: 0.9592 - lr: 1.0000e-04
Epoch 7/100
1856/1856 [==============================] - 366s 19